In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=rdHapuH5Tse8G26FaG56SBZQe_f1WKmWYK1A95Wk7uU&tc=aiXVLbksMRpBYTlH0DUdFQHRkjlTIEBtqY82lMhZO_k&cc=Dzglxjz-Nd6Y9D-4pVqAmYPUPtAcGWhhrfcAbXmO6lM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmlDjM9R6qSKL9z60o3ZXwM3-2zpIX6H1lQUUASBmfxwPh_LlEAvOU

Successfully saved authorization token.


In [ ]:
from google.colab import drive
path = '/content/drive/'
drive.mount(path)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
mi_geom = ee.FeatureCollection('projects/ee-anfrinconal/assets/AO_PR1')

In [ ]:
mi_geom_python = mi_geom.getInfo()
print(mi_geom)

ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Collection.loadTable",
    "arguments": {
      "tableId": {
        "constantValue": "projects/ee-anfrinconal/assets/AO_PR1"
      }
    }
  }
})


In [ ]:
# Importar archivos para preprocesamiento
import os
project_folder = '/content/drive/MyDrive/PRA_01/'
os.chdir(project_folder)


In [ ]:
import folium
# Crear un mapa interactivo con Folium
map = folium.Map(location=[0, 0], zoom_start=10)

# Agregar la geometría al mapa
folium.GeoJson(
    data=mi_geom_python,
    name='Mi Geometría',
).add_to(map)

# Mostrar el mapa en Colab
map.save('map.html')
map

In [ ]:
import wrapper as wp
#/***************************/
#// MAIN
#/***************************/
#Parameters
parameter = {  'START_DATE': '2018-01-01',
            'STOP_DATE': '2018-01-10',
            'POLARIZATION': 'VVVH',
            'PLATFORM_NUMBER' : 'A',
            'ORBIT' : 'DESCENDING',
            'ORBIT_NUM': None,
            'ROI': ee.Geometry.Rectangle([-47.1634, -3.00071, -45.92746, -5.43836]),
            'APPLY_BORDER_NOISE_CORRECTION': False,
            'APPLY_SPECKLE_FILTERING': True,
            'SPECKLE_FILTER_FRAMEWORK':'MULTI',
            'SPECKLE_FILTER': 'GAMMA MAP',
            'SPECKLE_FILTER_KERNEL_SIZE': 9,
            'SPECKLE_FILTER_NR_OF_IMAGES':10,
            'APPLY_TERRAIN_FLATTENING': True,
            'DEM': ee.Image('USGS/SRTMGL1_003'),
            'TERRAIN_FLATTENING_MODEL': 'VOLUME',
            'TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER':0,
            'FORMAT': 'DB',
            'CLIP_TO_ROI': True,
            'SAVE_ASSET': True,
            'ASSET_ID': "users/anfrinconal"
            }
#processed s1 collection
s1_processed = wp.s1_preproc(parameter)


Number of images in collection:  3
Multi-temporal speckle filtering is completed
Radiometric terrain normalization is completed
Exporting S1A_IW_GRDH_1SDV_20180107T084137_20180107T084202_020046_022281_87D2 to users/anfrinconal/S1A_IW_GRDH_1SDV_20180107T084137_20180107T084202_020046_022281_87D2
Exporting S1A_IW_GRDH_1SDV_20180107T084202_20180107T084227_020046_022281_1045 to users/anfrinconal/S1A_IW_GRDH_1SDV_20180107T084202_20180107T084227_020046_022281_1045
Exporting S1A_IW_GRDH_1SDV_20180107T084227_20180107T084252_020046_022281_EA84 to users/anfrinconal/S1A_IW_GRDH_1SDV_20180107T084227_20180107T084252_020046_022281_EA84


In [ ]:
import folium

In [ ]:




#collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
collection = s1_processed
# Filtrar la colección según tus criterios (puedes agregar filtros según sea necesario)
filtered_collection = collection.filterBounds(ee.Geometry.Point(-46.0, -3.5))



image_count = filtered_collection.size().getInfo()
print("Número de imágenes en la colección:", image_count)

# Obtener una lista de imágenes de la colección
image_list = filtered_collection.toList(image_count)

# Crear un mapa interactivo con Folium
map = folium.Map(location=[-3.5, -46.0], zoom_start=6)

# Agregar las imágenes de la colección al mapa
for i in range(image_count):
    image = ee.Image(image_list.get(i))
    image_url = image.getThumbURL({'bands': ['VV', 'VH'], 'min': -30, 'max': 0})
    folium.raster_layers.TileLayer(
        tiles=image_url,
        attr='Google Earth Engine',
        name=f'Imagen {i + 1}',
        overlay=True,
        control=True,
    ).add_to(map)

# Mostrar el mapa en Colab
map.save('map.html')
map

Número de imágenes en la colección: 1


EEException: ignored

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

In [ ]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
                -74.27909496,
                3.63869903
              ],
              [
                -74.23446728,
                3.63869903
              ],
              [
                -74.23446728,
                3.6015375
              ],
              [
                -74.27909496,
                3.6015375
              ],
              [
                -74.27909496,
                3.63869903
              ]
          ]
        ]
      }
    }
  ]
}

In [ ]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [ ]:
ffa_db.bandNames().getInfo()

['VV', 'VH', 'angle']

In [ ]:
sent_2 = ee.ImageCollection("COPERNICUS/S2_SR")
.filterBounds(aoi)



SyntaxError: ignored

In [ ]:
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD')
                       .filterBounds(aoi)
                       .filterDate(ee.Date('2016-01-01'), ee.Date('2018-12-31'))
                       .first()
                       .clip(aoi))
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                       .filterBounds(aoi)
                       .filterDate(ee.Date('2016-01-01'), ee.Date('2016-01-01'))
                       .first()
                       .clip(aoi))

In [ ]:
url = ffa_db.select('VV').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)

In [ ]:
ffa_db.getInfo()

NameError: ignored

In [ ]:
ffa_db.bandNames().getInfo()

['VV', 'VH', 'angle']

In [ ]:
import ee
import folium

# Iniciar la API de Earth Engine
ee.Initialize()

# Definir la colección de imágenes
collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')

# Filtrar la colección según tus criterios (puedes agregar filtros según sea necesario)
filtered_collection = collection.filterBounds(ee.Geometry.Point(-46.0, -3.5))

# Obtener el número de imágenes en la colección
image_count = filtered_collection.size().getInfo()
print("Número de imágenes en la colección:", image_count)

# Obtener una lista de imágenes de la colección
image_list = filtered_collection.toList(image_count)

# Crear un mapa interactivo con Folium
map = folium.Map(location=[-3.5, -46.0], zoom_start=6)

# Agregar las imágenes de la colección al mapa
for i in range(image_count):
    image = ee.Image(image_list.get(i))
    image_url = image.getThumbURL({'bands': ['VV', 'VH']})#, 'min': -30, 'max': 0})
    folium.raster_layers.TileLayer(
        tiles=image_url,
        attr='Google Earth Engine',
        name=f'Imagen {i + 1}',
        overlay=True,
        control=True,
    ).add_to(map)

# Mostrar el mapa en Colab
map.save('map.html')
map


Número de imágenes en la colección: 211


EEException: ignored